In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/data (3).csv')
df.drop('genre_list', axis = 1)

,level_0,index,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,vote_average,vote_count,credits,keywords,success,release_year
0,0,0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,An exploratory dive into the deepest depths of...,8763.998,['Apelles Entertainment-Warner Bros. Pictures-...,2023-08-02,129000000.0,3.520565e+08,116.0,Released,7.079,1365.0,Jason Statham-Wu Jing-Shuya Sophia Cai-Sergio ...,based on novel or book-sequel-kaiju,Successful,2023
1,1,1,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,Father Gabriele Amorth Chief Exorcist of the V...,5953.227,['Screen Gems-2.0 Entertainment-Jesus & Mary-W...,2023-04-05,18000000.0,6.567582e+07,103.0,Released,7.433,545.0,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco...,spain-rome italy-vatican-pope-pig-possession-c...,Successful,2023
2,2,2,533535,Deadpool & Wolverine,Action-Comedy-Science Fiction,en,A listless Wade Wilson toils away in civilian ...,5410.496,['Marvel Studios-Maximum Effort-21 Laps Entert...,2024-07-24,200000000.0,1.326387e+09,128.0,Released,7.765,3749.0,Ryan Reynolds-Hugh Jackman-Emma Corrin-Matthew...,hero-superhero-anti hero-mutant-breaking the f...,Successful,2024
3,3,3,667538,Transformers: Rise of the Beasts,Action-Adventure-Science Fiction,en,When a new threat capable of destroying the en...,5409.104,['Skydance-Paramount-di Bonaventura Pictures-B...,2023-06-06,200000000.0,4.070455e+08,127.0,Released,7.340,1007.0,Anthony Ramos-Dominique Fishback-Luna Lauren V...,peru-alien-end of the world-based on cartoon-b...,Successful,2023
4,4,4,693134,Dune: Part Two,Science Fiction-Adventure,en,Follow the mythic journey of Paul Atreides as ...,4742.163,['Legendary Pictures'],2024-02-27,190000000.0,6.838137e+08,167.0,Released,8.300,2770.0,Timothée Chalamet-Zendaya-Rebecca Ferguson-Jav...,epic-based on novel or book-fight-sandstorm-sa...,Successful,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8216,45177,198982,173336,Il nostro matrimonio è in crisi,Comedy,it,Unknown,1.152,['Filmauro'],2001-12-31,4000000.0,1.500000e+06,0.0,Released,5.800,22.0,Antonio Albanese-Aisha Cerami-Dino Abbrescia-S...,Unknown,Not Successful,2001
8217,45275,286877,116741,The Internship,Comedy,en,Two recently laid-off men in their 40s try to ...,0.822,['20th Century Fox-Regency Enterprises-21 Laps...,2013-06-07,58000000.0,4.400000e+07,119.0,Released,6.300,3781.0,Vince Vaughn-Owen Wilson-Rose Byrne-Aasif Mand...,competition-california-mattress shop-job inter...,Not Successful,2013
8218,45293,315257,203793,Stalingrad,War-Action-Drama,ru,A band of determined Russian soldiers fight to...,0.656,['Art Pictures Studio-Non-Stop Productions'],2013-10-09,30000000.0,6.812952e+07,131.0,Released,6.500,289.0,Thomas Kretschmann-Yanina Studilina-Philippe R...,world war ii-soviet union-nazi-soviet military...,Successful,2013
8219,45305,436761,188640,Shootout at Wadala,Action-Crime-Drama-History,hi,The first-ever registered encounter by the Mum...,0.600,['Balaji Motion Pictures'],2013-05-03,6900000.0,1.100000e+07,150.0,Released,5.923,26.0,John Abraham-Anil Kapoor-Kangana Ranaut-Tussha...,criminal underworld,Not Successful,2013


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8221 entries, 0 to 8220
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   level_0               8221 non-null   int64  
 1   index                 8221 non-null   int64  
 2   id                    8221 non-null   int64  
 3   title                 8221 non-null   object 
 4   genres                8221 non-null   object 
 5   original_language     8221 non-null   object 
 6   overview              8221 non-null   object 
 7   popularity            8221 non-null   float64
 8   production_companies  8221 non-null   object 
 9   release_date          8221 non-null   object 
 10  budget                8221 non-null   float64
 11  revenue               8221 non-null   float64
 12  runtime               8221 non-null   float64
 13  status                8221 non-null   object 
 14  vote_average          8221 non-null   float64
 15  vote_count           

In [ ]:
df['genre_list'] = df['genres'].apply(lambda x: x.split('-'))
print(df['genre_list'])

0          [Action, Science Fiction, Horror]
1                [Horror, Mystery, Thriller]
2          [Action, Comedy, Science Fiction]
3       [Action, Adventure, Science Fiction]
4               [Science Fiction, Adventure]
                        ...                 
8216                                [Comedy]
8217                                [Comedy]
8218                    [War, Action, Drama]
8219         [Action, Crime, Drama, History]
8220                 [Action, Comedy, Drama]
Name: genre_list, Length: 8221, dtype: object


In [ ]:
from gensim.models import Word2Vec
genre_sentences = df['genre_list'].tolist()
model = Word2Vec(sentences=genre_sentences, vector_size=50, window=5, min_count=1, sg=1)
def get_movie_embedding(genres, model):
    genre_vectors = [model.wv[genre] for genre in genres if genre in model.wv]
    if genre_vectors:
        return np.mean(genre_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

df['genre_embedding'] = df['genre_list'].apply(lambda genres: get_movie_embedding(genres, model))
embedding_df = pd.DataFrame(df['genre_embedding'].tolist(), index=df.index)
df = pd.concat([df, embedding_df], axis=1)

df = df.drop(columns=['genres', 'genre_list', 'genre_embedding'])


In [ ]:
model.save("word2vec_genres.model")


In [ ]:
df = df.drop(columns=['id','index','keywords','title','original_language','overview','status','success'])
df.columns = df.columns.astype('str')

df['release_month'] = pd.to_datetime(df['release_date']).dt.month
df = df.drop(columns=['release_date','release_year'])

from sklearn.preprocessing import LabelEncoder

df['production_companies'] = df['production_companies'].astype('str')
df['production_companies'] = df['production_companies'].str.split('-')

companies_exploded = df.explode('production_companies')

le = LabelEncoder()
companies_exploded['production_companies_encoded'] = le.fit_transform(companies_exploded['production_companies'])

df = df.drop(columns=['production_companies']).join(
    companies_exploded[['production_companies_encoded']].groupby(level=0).sum()
)



import joblib
joblib.dump(le, 'company_encoders.pkl')

['company_encoders.pkl']

In [ ]:
df['credits'] = df['credits'].astype('str')
df['credits'] = df['credits'].str.split('-')

credits_exploded = df.explode('credits')

le1 = LabelEncoder()
credits_exploded['credits_encoded'] = le1.fit_transform(credits_exploded['credits'])

df = df.drop(columns=['credits']).join(
    credits_exploded[['credits_encoded']].groupby(level=0).sum()
)

joblib.dump(le1, 'credits_encoders.pkl')


['credits_encoders.pkl']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [ ]:
X = df.drop(columns=['revenue'])
y = df['revenue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# model1 = XGBRegressor(n_estimators=100, learning_rate=0.07, max_depth=6, enable_categorical=True, random_state=42)
# model1.fit(X_train, y_train)

# train_score = model1.score(X_train, y_train)
# test_score = model1.score(X_test, y_test)

# print("Acc:", train_score)
# print("Test Score:", test_score)

# from sklearn.ensemble import RandomForestRegressor


model1 = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42)

model1.fit(X_train, y_train)

train_score = model1.score(X_train, y_train)
test_score = model1.score(X_test, y_test)

In [ ]:
train_score, test_score

(0.9601653434765999, 0.7578809519734527)

In [ ]:
import joblib

# Save the XGBoost model
joblib.dump(model1, 'rgb_model.pkl')


['rgb_model.pkl']

In [ ]:
df.head()

,level_0,popularity,budget,revenue,runtime,vote_average,vote_count,0,1,2,...,43,44,45,46,47,48,49,release_month,production_companies_encoded,credits_encoded
0,0,8763.998,129000000.0,3.520565e+08,116.0,7.079,1365.0,-0.090179,0.002271,-0.004648,...,-0.174168,0.314167,0.148310,0.089899,-0.084048,0.088124,-0.060161,8,35904,2452220
1,1,5953.227,18000000.0,6.567582e+07,103.0,7.433,545.0,-0.068435,-0.016345,-0.000814,...,-0.178240,0.304312,0.145999,0.085017,-0.088351,0.075450,-0.056062,4,31511,2136074
2,2,5410.496,200000000.0,1.326387e+09,128.0,7.765,3749.0,-0.103017,0.005946,-0.000635,...,-0.179493,0.329392,0.158959,0.081470,-0.088134,0.095238,-0.052769,7,27719,4162250
3,3,5409.104,200000000.0,4.070455e+08,127.0,7.340,1007.0,-0.098592,0.001999,-0.003790,...,-0.185639,0.333820,0.160529,0.095385,-0.082089,0.103003,-0.065258,6,47117,3204041
4,4,4742.163,190000000.0,6.838137e+08,167.0,8.300,2770.0,-0.095297,-0.000664,-0.014430,...,-0.183784,0.342783,0.153561,0.095199,-0.079565,0.107733,-0.071637,2,10466,7669262


In [ ]:
sample = {'level_0': 100, 'popularity': 5000.0, 'budget': 129000000.0, 'runtime': 125.0,'vote_average': 8.0, 'vote_count': 1365.0, 'genres':['Action','Thriller'],'release_month': 5, 'production_companies':['Legendary Pictures'], 'credits':['Russell Crowe-Daniel Zovatto']}